In [3]:
import pandas as pd
import numpy as np
import joblib

In [4]:
# Modelos y transformadores
modelo = joblib.load('../model/modelo_multiclase_v2.pkl')
scaler = joblib.load('../model/scaler_full.pkl')
cat_encoder = joblib.load('../model/encoder_categoricos.pkl')
label_encoder = joblib.load('../model/encoder_etiquetas.pkl')

In [5]:
df_base = pd.read_csv('../data/teclados_limpio.csv')
df_base.fillna(0, inplace=True)

In [11]:
def predecir_categoria(teclado_nombre, precio_usuario):
    fila = df_base[df_base["Name"] == teclado_nombre]

    if fila.empty:
        return f"❌ Teclado '{teclado_nombre}' no encontrado en la base de datos."

    fila = fila.iloc[0]
    tipo = fila.get("Type", "")
    conexion = fila.get("Connection", "")
    switches = fila.get("Switches", "")
    rating = fila.get("Rating", 0)
    stores = fila.get("Stores", 0)

    # Escalar datos numéricos con nombres
    num_df = pd.DataFrame([[precio_usuario, rating, stores]], columns=["SimulatedPrice", "Rating", "Stores"])
    num_features = scaler.transform(num_df)

    # Codificar datos categóricos con nombres
    cat_df = pd.DataFrame([[teclado_nombre, tipo, conexion, switches]],
                        columns=["Name", "Type", "Connection", "Switches"])
    cat_features = cat_encoder.transform(cat_df)

    X_input = np.hstack([num_features, cat_features])

    y_pred = modelo.predict(X_input)
    etiqueta = label_encoder.inverse_transform(y_pred)[0]

    return {
        "Teclado": teclado_nombre,
        "Precio ingresado": precio_usuario,
        "Precio real": fila["Price"],
        "Categoría": etiqueta
    }


In [12]:
print(predecir_categoria("Logitech G915 TKL (GL Clicky)", 185.00))
print(predecir_categoria("Redragon Yama", 39.99))
print(predecir_categoria("Corsair K63", 150.00))

{'Teclado': 'Logitech G915 TKL (GL Clicky)', 'Precio ingresado': 185.0, 'Precio real': np.float64(229.0), 'Categoría': 'BuenPrecio'}
{'Teclado': 'Redragon Yama', 'Precio ingresado': 39.99, 'Precio real': np.float64(86.0), 'Categoría': 'Oferton'}
{'Teclado': 'Corsair K63', 'Precio ingresado': 150.0, 'Precio real': np.float64(99.99), 'Categoría': 'Estafa'}
